# analyze_CPP_Rg
Compute CPPRg for result_LM_1 trajectories

In [ ]:
import csv
import glob
import pandas as pd
import numpy as np
import MDAnalysis as mda

In [ ]:
def compute_cpprg(psf_path, traj_pattern, atom_dict_csv):
    """
    Compute the critical packing parameter (CPPRg) as the ratio of the
    radius of gyration of tail atoms to head atoms, averaged over a trajectory.

    Parameters
    ----------
    psf_path : str
        Path to the PSF structure file.
    traj_pattern : str
        Glob pattern matching the trajectory files (e.g., '*.dcd').
    atom_dict_csv : str
        Path to a CSV file with columns:
            'resname', 'head_atoms', 'tail_atoms'
        where head_atoms and tail_atoms are semicolon-separated atom names.

    Returns
    -------
    mean_ratio : float
        The mean value of Rg_tail / Rg_head over all frames.
    sd_ratio : float
        The standard deviation of Rg_tail / Rg_head.
    """
    # 1) Load head/tail atom mapping from CSV
    atom_map = []
    with open(atom_dict_csv) as f:
        reader = csv.DictReader(f)
        for row in reader:
            atom_map.append({
                'resname': row['resname'],
                'head':    row['head_atoms'].split(';'),
                'tail':    row['tail_atoms'].split(';')
            })

    # 2) Initialize MDAnalysis Universe and collect Rg ratios per frame
    u = mda.Universe(psf_path, *mda.glob(traj_pattern))
    ratios = []
    for ts in u.trajectory:
        heads, tails = [], []
        # Gather head and tail coordinates for each residue
        for m in atom_map:
            sel_h = u.select_atoms(
                f"resname {m['resname']} and name {' '.join(m['head'])}"
            )
            sel_t = u.select_atoms(
                f"resname {m['resname']} and name {' '.join(m['tail'])}"
            )
            if sel_h.n_atoms:
                heads.append(sel_h.positions)
            if sel_t.n_atoms:
                tails.append(sel_t.positions)
        # If both head and tail atoms are present, compute Rg ratio
        if heads and tails:
            rg_h = radius_of_gyration(np.vstack(heads))
            rg_t = radius_of_gyration(np.vstack(tails))
            ratios.append(rg_t / rg_h)

    # 3) Compute statistics on the collected ratios
    arr = np.array(ratios)
    return arr.mean(), arr.std()


In [ ]:
# 1) Parameters
psf_file    = '/data/structure/result_LM_1.psf'
atom_dict   = '/data/misc/il_atom_dict_LM_2019.csv'
traj_specs  = [
    ('protonated', '/data/MD_trajectory_protonated/result_LM_1/*.dcd'),
    ('neutral',    '/data/MD_trajectory_neutral/result_LM_1/*.dcd')
]

# 2) Compute CPPRg for each condition
results = []
for label, traj_pattern in traj_specs:
    mean, sd = compute_cpprg(
        psf_file,
        traj_pattern,
        atom_dict,
        mode='terminal',    # or 'total'
        resname='LMBIC',
        time_range=(1000, 1500)
    )
    results.append({
        'Formulation':   'result_LM_1',
        'Label':         label,
        'CPPRg':         mean,
        'CPPRg_sd':      sd
    })

# 3) Write out CSVs
df = pd.DataFrame(results)
df[df.Label=='protonated']\
  .drop(columns='Label')\
  .to_csv('results/CPP_Rg_results_protonated.csv', index=False)
df[df.Label=='neutral']\
  .drop(columns='Label')\
  .to_csv('results/CPP_Rg_results_neutral.csv',     index=False)